In [4]:
import pandas as pd
from os import walk
import pyarrow.parquet as parquet
from collections import defaultdict
import datetime
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import numpy as np
import scipy.spatial.distance
import re
import math
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from multiprocessing import Pool, cpu_count
import mroc
from nltk.cluster import KMeansClusterer
import nltk

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

# Where the downloaded data are
input_path = '/Users/tyamgin/Projects/mlbootcamp/championship19/data'
# Where to store results
output_path = '/Users/tyamgin/Projects/mlbootcamp/championship19/res'

In [2]:
cpu_count()

8